<a href="https://colab.research.google.com/github/ignacioperezallub/ml_tensorflow_fcc/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
   #%tensorflow_version only exists in Colab.
   #!pip install tf-nightly
  pass
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence

print(tf.__version__)

2.11.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])

train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]

train_labels =  train_df["type"].values
train_ds = tf.data.Dataset.from_tensor_slices((train_df["msg"].values, train_labels))

test_labels =  test_df["type"].values
test_ds = tf.data.Dataset.from_tensor_slices((test_df["msg"].values, test_labels))


In [ ]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

vector_text = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vector_text.adapt(train_ds.map(lambda text, label: text))
vocab = np.array(vector_text.get_vocabulary())

In [ ]:
model = tf.keras.Sequential([
    vector_text,
    tf.keras.layers.Embedding(
        len(vector_text.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=12,
)

Epoch 1/12
44/44 [==============================] - 192s 4s/step - loss: 0.0284 - accuracy: 0.9935 - val_loss: 0.0320 - val_accuracy: 0.9937
Epoch 2/12
44/44 [==============================] - 192s 4s/step - loss: 0.0272 - accuracy: 0.9928 - val_loss: 0.0262 - val_accuracy: 0.9958
Epoch 3/12
44/44 [==============================] - 192s 4s/step - loss: 0.0242 - accuracy: 0.9957 - val_loss: 0.0247 - val_accuracy: 0.9958
Epoch 4/12
44/44 [==============================] - 192s 4s/step - loss: 0.0231 - accuracy: 0.9950 - val_loss: 0.0220 - val_accuracy: 0.9958
Epoch 5/12
44/44 [==============================] - 194s 4s/step - loss: 0.0197 - accuracy: 0.9950 - val_loss: 0.0226 - val_accuracy: 0.9958
Epoch 6/12
44/44 [==============================] - 176s 4s/step - loss: 0.0198 - accuracy: 0.9957 - val_loss: 0.0189 - val_accuracy: 0.9969
Epoch 7/12
44/44 [==============================] - 195s 4s/step - loss: 0.0150 - accuracy: 0.9978 - val_loss: 0.0178 - val_accuracy: 0.9969
Epoch 8/12
44

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

44/44 [==============================] - 36s 825ms/step - loss: 0.0102 - accuracy: 0.9978
Test Loss: 0.010211157612502575
Test Accuracy: 0.9978448152542114


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict([pred_text])
  print(prediction)
  p = prediction[0][0]
  return [p, "ham" if p <0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 1s 554ms/step
[[-7.5385137]]
[-7.5385137, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 1s 1s/step
[[-7.5385137]]
[-7.5385137, 'ham']
1/1 [==============================] - 1s 1s/step
[[1.3212322]]
[1.3212322, 'spam']
1/1 [==============================] - 1s 916ms/step
[[-12.617896]]
[-12.617896, 'ham']
1/1 [==============================] - 1s 727ms/step
[[0.4734715]]
[0.4734715, 'ham']
1/1 [==============================] - 1s 704ms/step
[[4.741449]]
[4.741449, 'spam']
1/1 [==============================] - 1s 746ms/step
[[-12.431992]]
[-12.431992, 'ham']
1/1 [==============================] - 1s 979ms/step
[[-11.868933]]
[-11.868933, 'ham']
You haven't passed yet. Keep trying.
